In [7]:
%pip install ../..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=87867 sha256=5e7e21e03e754c2dbbba4567bde7ada6f8de1198b13f8f607cb5cee149b22f4d
  Stored in directory: /tmp/pip-ephem-wheel-cache-01rf5nl0/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch 

device = torch.device("cuda:7")

In [9]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

In [10]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [11]:
from spuco.models import model_factory 
from spuco.utils import Trainer
from torch.optim import SGD

model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)
trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-5, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
trainer.train(1)

Epoch 0: 100%|██████████| 751/751 [00:08<00:00, 87.11batch/s, accuracy=100.0%, loss=1.47]   


In [13]:
from spuco.group_inference import EIIL

logits = trainer.get_trainset_outputs()
eiil = EIIL(
    logits=logits,
    class_labels=trainset.labels,
    num_steps=100, 
    lr=1e-1,
    device=device,
    verbose=True
)

Getting Trainset Outputs: 100%|██████████| 751/751 [00:04<00:00, 183.44batch/s]


In [14]:
group_partition = eiil.infer_groups()

EIIL Inferring Groups: 100%|██████████| 100/100 [00:00<00:00, 674.53it/s]


In [15]:
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

(0, 0) 10090
(0, 1) 43
(1, 0) 9635
(1, 1) 37
(2, 0) 8972
(2, 1) 39
(3, 0) 9720
(3, 1) 27
(4, 0) 9414
(4, 1) 27


In [16]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:  10%|█         | 1/10 [00:01<00:09,  1.09s/it]

Group (0, 0) Test Accuracy: 99.9207135777998


Evaluating group-wise accuracy:  20%|██        | 2/10 [00:01<00:05,  1.41it/s]

Group (0, 1) Test Accuracy: 0.0


Evaluating group-wise accuracy:  30%|███       | 3/10 [00:02<00:06,  1.16it/s]

...

Evaluating group-wise accuracy:  90%|█████████ | 9/10 [00:07<00:00,  1.11it/s]

Group (4, 0) Test Accuracy: 99.77692797960485


Evaluating group-wise accuracy: 100%|██████████| 10/10 [00:07<00:00,  1.25it/s]

Group (4, 1) Test Accuracy: 0.0


{(0, 0): 99.9207135777998,
 (0, 1): 0.0,
 (1, 0): 99.86507524649714,
 (1, 1): 2.7027027027027026,
 (2, 0): 36.35755684351315,
 (2, 1): 0.0,
 (3, 0): 99.84567901234568,
 (3, 1): 0.0,
 (4, 0): 99.77692797960485,
 (4, 1): 0.0}

In [17]:
from torch.optim import SGD
from spuco.invariant_train import GroupDRO
from spuco.models import model_factory 
from spuco.datasets import GroupLabeledDatasetWrapper

group_labeled_trainset = GroupLabeledDatasetWrapper(trainset, group_partition)
model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)

group_dro = GroupDRO(
    model=model,
    num_epochs=1,
    trainset=group_labeled_trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

In [18]:
group_dro.train()

Epoch 0: 100%|██████████| 751/751 [00:11<00:00, 63.73batch/s, accuracy=50.0%, loss=0.278]   


In [19]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:11,  2.02it/s]

Group (0, 0) Test Accuracy: 71.39479905437352


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:01<00:11,  1.94it/s]

Group (0, 1) Test Accuracy: 32.38770685579196


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:11,  1.87it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:13<00:00,  1.81it/s]

Group (4, 3) Test Accuracy: 33.83838383838384


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:13<00:00,  1.84it/s]

Group (4, 4) Test Accuracy: 97.97979797979798


{(0, 0): 71.39479905437352,
 (0, 1): 32.38770685579196,
 (0, 2): 86.05200945626477,
 (0, 3): 59.33806146572104,
 (0, 4): 0.0,
 (1, 0): 53.30073349633252,
 (1, 1): 78.239608801956,
 (1, 2): 1.2254901960784315,
 (1, 3): 77.94117647058823,
 (1, 4): 47.549019607843135,
 (2, 0): 90.93333333333334,
 (2, 1): 3.466666666666667,
 (2, 2): 75.2,
 (2, 3): 28.0,
 (2, 4): 7.4866310160427805,
 (3, 0): 57.78894472361809,
 (3, 1): 34.5088161209068,
 (3, 2): 0.0,
 (3, 3): 89.67254408060454,
 (3, 4): 5.037783375314861,
 (4, 0): 0.0,
 (4, 1): 77.58186397984886,
 (4, 2): 59.44584382871537,
 (4, 3): 33.83838383838384,
 (4, 4): 97.97979797979798}

In [20]:
evaluator.worst_group_accuracy

((0, 4), 0.0)

In [21]:
evaluator.average_accuracy

82.21385450809817

In [22]:
evaluator.evaluate_spurious_task()

43.46